In [1]:
import sys

sys.path.append("..")
from typing import List

import requests
from bs4 import BeautifulSoup
from googlesearch import search
from pydantic import BaseModel, Field
from rich import print
from brain.sdk import BrainClient

In [2]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
from typing import List, Union

from pydantic import BaseModel, Field


class Tool(BaseModel):
    script: str = Field(..., description="The Python script to be executed. Ensure it handles errors and outputs structured results.")
    pip_install: List[str] = Field(..., description="The list of pip packages required to execute the script.")
    completed: Union[bool, str] = Field(..., description="Indicates if the task is successfully completed. True if completed, otherwise a string explaining why the script cannot be generated.")

In [4]:
@brain_client.reasoner(schema=Tool)
def autonomous_tool(task: str):
    """
    Generates standalone, executable Python scripts with no external dependencies.
    """
    system_prompt = """You are a Python script generator focused on creating fully autonomous code. Your scripts must be completely self-contained:

DO NOT generate scripts that:
- Require external inputs (API keys, file paths, user prompts, etc.)
- Use mock/placeholder data
- Need manual configuration
- Access local files or environment variables
- Connect to external services without built-in test data
- Depend on system-specific resources
- Use unreliable or niche packages

Instead:
- Use standard library when possible
- Include test data within the script
- Handle all edge cases and errors
- Use only widely-used PyPI packages if needed
- Generate any required data programmatically
- If we cannot do it without external dependencies, explain why in 'completed' field and guide the tool maker to say it cannot be done

If task requires unavoidable external dependencies, explain why in 'completed' field."""

    user_prompt = f"""Generate a standalone Python script for: "{task}"

Return:
- script: Fully executable Python code
- pip_install: Required packages (empty list if none needed)
- completed: True if successful, or explanation why task cannot be autonomous"""

    return system_prompt, user_prompt
autonomous_tool_id=autonomous_tool.register()

In [5]:
import os
import shutil
import subprocess
import tempfile
import venv
from typing import Dict, Union

def execute_script_with_temp_venv(script: str, pip_install: list) -> Dict[str, Union[str, bool]]:
    result = {
        "script": script,
        "pip_install": pip_install,
        "success": False,
        "stdout": "",
        "stderr": "",
        "error": ""
    }
    temp_dir = None

    try:
        # Step 1: Create a temporary directory for the virtual environment
        temp_dir = tempfile.mkdtemp()
        venv_dir = os.path.join(temp_dir, "venv")
        venv.create(venv_dir, with_pip=True)

        # Step 2: Paths for the virtual environment
        venv_python = os.path.join(venv_dir, "bin", "python") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "python")
        venv_pip = os.path.join(venv_dir, "bin", "pip") if os.name != 'nt' else os.path.join(venv_dir, "Scripts", "pip")

        # Step 3: Install required packages
        if pip_install:
            install_command = [venv_pip, "install"] + pip_install
            subprocess.run(install_command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Step 4: Write the script to a temporary file
        script_path = os.path.join(temp_dir, "script.py")
        with open(script_path, "w") as script_file:
            script_file.write(script)

        # Step 5: Execute the script
        execute_command = [venv_python, script_path]
        process = subprocess.run(
            execute_command, check=False, capture_output=True, text=True
        )

        # Step 6: Update result based on execution outcome
        result["stdout"] = process.stdout.strip()
        result["stderr"] = process.stderr.strip()
        result["error"] = ""
        result["success"] = process.returncode == 0 and not result["stderr"]

    except subprocess.CalledProcessError as e:
        result["error"] = f"Pip installation or execution failed: {str(e)}"
    except Exception as e:
        result["error"] = f"Unexpected error: {str(e)}"
    finally:
        # Step 7: Cleanup temporary directory
        if temp_dir and os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

    # Update success condition to ensure no stderr or unexpected errors
    result["success"] = result["success"] and not result["error"] and not result["stderr"]
    return result

def format_tool_output(execution_result: Dict[str, Union[str, bool]]) -> str:
    if execution_result["success"]:
        return f"Tool executed successfully. Output:\n{execution_result['stdout']}\n"
    else:
        return (
            f"Tool execution failed.\n"
            f"Error: {execution_result['error'] if execution_result['error'] else 'No additional error details.'}\n"
            f"Standard Error: {execution_result['stderr']}\n"
            f"Standard Output: {execution_result['stdout']}\n"
            f"Script:\n{execution_result['script']}\n"
            f"Dependencies:\n{execution_result['pip_install']}\n"
        )



In [6]:
from typing import Optional, Union

from pydantic import BaseModel, Field


class Success(BaseModel):
    final_answer: str = Field(..., description="The final result or answer to the task if the execution succeeded.")

class Failure(BaseModel):
    analysis: str = Field(..., description="A detailed explanation of why the tool execution failed.")
    suggestions: str = Field(..., description="Suggestions for fixing the issue or alternative approaches to achieve the task.")

class Result(BaseModel):
    success: bool = Field(..., description="Indicates if the tool execution was successful.")
    result: Union[Success, Failure] = Field(..., description="The result of the analysis, including success or failure details.")
    
@brain_client.reasoner(schema=Result)
def result_analyzer(tool_output: str, task: str):
    """
    Analyzes script execution results focusing on zero human intervention requirement.
    """
    system_prompt = """Analyze Python script results for true autonomy - script must run without any human intervention.

Success case:
- Mention success as True
Extract meaningful result from tool_output and present as:
- Direct answer addressing the original task
- Natural, human-readable format
- No technical details unless relevant
- No mention of script/code aspects


Failure case - Check for:
1. Manual Requirements (like these, but it can be more):
   - API keys/credentials needed
   - File paths/names input
   - User prompts/inputs
   - Configuration steps
   - Runtime user decisions

2. Script should:
   - Use standard library or pip packages
   - Make API calls if no credentials needed
   - Handle all data programmatically
   - Run start-to-finish autonomously

Key principle:
If task requires ANY manual steps (API keys, inputs, etc):
- Clearly state why it can't be autonomous
- Don't suggest mock/placeholder solutions
- Only suggest truly automatic alternatives"""

    user_prompt = f"""Analyze result for: "{task}"
Output: "{tool_output}"

Success: Direct task response
Failure: Explain manual intervention needs"""

    return system_prompt, user_prompt
result_analyzer_id = result_analyzer.register()

In [7]:
@brain_client.multi_agent
def autonomous_tool(task: str, max_trials=5, verbose=True):
    for i in range(max_trials):
        print(f"{'='*10} Iteration {i+1} {'='*10}")
        
        if i != 0:
            if not res.success:
                task_context = (
                    f"Last tool failed because {res.result}. "
                    f"Last tool script:\n{tool_output.script}\n"
                    f"Pip dependencies: {tool_output.pip_install}\n"
                )
        else:
            task_context = ""

        # Generate the tool
        tool_output = brain_client.use(autonomous_tool_id)(task=task_context + task)
        
        # Execute the script
        exec_output = execute_script_with_temp_venv(tool_output.script, tool_output.pip_install)
        
        # Format execution result into context
        context = format_tool_output(exec_output)
        
        # Analyze the result
        res = brain_client.use(result_analyzer_id)(tool_output=context, task=task)
        
        if verbose:
            print(res.result)
        
        if res.success:
            if verbose:
                print(f"Final Tool Output:\n{context}")
            break
    
    return res.result


In [12]:
task="What is the temperature in Downtown Toronto today?"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Success(final_answer='The temperature in Downtown Toronto today is 24.0 degrees Celsius.')

Final Tool Output:
Tool executed successfully. Output:
{
    "location": "Downtown Toronto",
    "temperature_c": 24.0,
    "unit": "Celsius",
    "timestamp": "2024-11-27T12:21:12.965505"
}

Final Result: final_answer='The temperature in Downtown Toronto today is 24.0 degrees Celsius.'

In [8]:
task="search for top 3 articles on autonomous driving and give back the titles"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Failure(
    analysis="The script execution failed because it encountered a '429 Client Error' when attempting to make an 
API call. This error indicates that the script's requests to the API exceeded the allowed limit, which could 
suggest that the API requires a valid key or has rate-limiting constraints in place. Additionally, the error does 
not imply that the script can operate completely autonomously since it relies on an external API for data access, 
which may require configuration or credentials to function correctly.",
    suggestions='To ensure true autonomy, consider using an API that does not require credentials or has higher 
limits for requests, or find a different data source that can provide the needed information without manual 
intervention. Alternatively, implement error handling to manage rate limits more gracefully.'
)

========== Iteration 2 ==========

Success(
    final_answer='The search for top articles on autonomous driving returned the following summaries:\n\n1. **The 
Future of Autonomous Vehicles**: This publication predicts the trends in autonomous vehicle technology over the 
next decade.\n\n2. **Challenges in Autonomous Driving**: In this article, we discuss the obstacles faced in 
implementing self-driving cars in real-world scenarios.\n\n3. **Understanding Autonomous Driving Systems**: This 
paper explores the various technologies behind autonomous vehicles.'
)

Final Tool Output:
Tool executed successfully. Output:
{
  "articles": [
    {
      "title": "The Future of Autonomous Vehicles",
      "abstract": "This publication predicts the trends in autonomous vehicle technology over the next decade."
    },
    {
      "title": "Challenges in Autonomous Driving",
      "abstract": "In this article, we discuss the obstacles faced in implementing self-driving cars in real-world 
scenarios."
    },
    {
      "title": "Understanding Autonomous Driving Systems",
      "abstract": "This paper explores the various technologies behind autonomous vehicles."
    }
  ]
}

Final Result: final_answer='The search for top articles on autonomous driving returned the following 
summaries:\n\n1. **The Future of Autonomous Vehicles**: This publication predicts the trends in autonomous vehicle 
technology over the next decade.\n\n2. **Challenges in Autonomous Driving**: In this article, we discuss the 
obstacles faced in implementing self-driving cars in real-world scenarios.\n\n3. **Understanding Autonomous Driving
Systems**: This paper explores the various technologies behind autonomous vehicles.'

In [9]:
task="calculate the sum of numbers from 1 to 100_0000_000"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Success(final_answer='The sum of numbers from 1 to 100,000,000 is 5,000,000,500,000,000.')

Final Tool Output:
Tool executed successfully. Output:
The sum of numbers from 1 to 100000000 is: 5000000050000000

Final Result: final_answer='The sum of numbers from 1 to 100,000,000 is 5,000,000,500,000,000.'

In [10]:
task="Multiply 777_82 times 67625_7762_8"
result=autonomous_tool(task,max_trials=5,verbose=True)

print(f"Final Result: {result}")

========== Iteration 1 ==========

Success(final_answer='The multiplication of 77782 and 6762577628 results in 526006813061096.')

Final Tool Output:
Tool executed successfully. Output:
{'result': 526006813061096, 'number1': 77782, 'number2': 6762577628}

Final Result: final_answer='The multiplication of 77782 and 6762577628 results in 526006813061096.'

In [11]:
print(777_82*67625_7762_8)

526006813061096